In [1]:
import IPython.display
import tempfile
import os

In [2]:
import momapy.sbgn.io
import momapy.rendering
import momapy.builder
import momapy.coloring

In [3]:
def display(map_, width=400):
    f, filename = tempfile.mkstemp(dir=".", suffix=".svg")
    momapy.rendering.render_map(map_, filename, renderer="svg-native")
    s = "<img src='{}' width={}>".format(os.path.split(filename)[-1], width)
    IPython.display.display(IPython.display.HTML(s))

In [4]:
m1 = momapy.sbgn.io.read_file("phospho1.sbgn")
display(m1, 400)

In [5]:
m2 = momapy.sbgn.io.read_file("phospho2.sbgn")
display(m2, 300)

In [6]:
m1 == m2

False

In [7]:
m1.layout == m2.layout

False

In [8]:
m1.model == m2.model

True

In [9]:
try:
    m1.layout.stroke = momapy.coloring.colors.red
except Exception as e:
    print(e)

cannot assign to field 'stroke'


In [10]:
m1_builder = momapy.builder.builder_from_object(m1)

In [11]:
m1_builder.layout.stroke = momapy.coloring.colors.red
m1_builder.layout.stroke_width = 2
display(m1_builder, 400)

In [12]:
m1_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)

In [13]:
m1_builder.layout.stroke = momapy.coloring.colors.red
m1_builder.layout.stroke_width = 2
display(m1_builder, 400)

### So what is the purpose of the non builder object?

In [14]:
m1a_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m1b_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m1a_builder == m1b_builder

False

In [15]:
m1a = m1a_builder.build()
m1b = m1b_builder.build()
m1a == m1b

True

In [ ]:
s = set([])
for i in range(0, 255):
    for j in range(0, 255):
        for k in range(0, 255):
            m1a_builder.layout.stroke = momapy.coloring.rgb(i, j, k)
            s.add(m1a_builder.build())

In [ ]:
m1a in s